# Problem Objective

**To predict the optimal pricing for airline tickets to maximize revenue, utilizing flight data and demand patterns.**


## Step 1: Creating a Synthetic Dataset
For our first step, we'll create a synthetic dataset to simulate the conditions and data you'd encounter in a real-world scenario involving airline pricing. This dataset will include features such as flight date, time of day, origin and destination airports, current ticket price, number of seats available, and an estimation of demand level. These factors are critical in determining optimal pricing strategies for maximizing revenue.

We'll generate data for 10,000 flights with the following attributes:

- Flight ID: Unique identifier for each flight.
- Date: Date of the flight.
- Day of Week: The day of the week the flight occurs.
- Time of Day: Categorized as Morning, Afternoon, or Evening.
- Duration: Duration of the flight in hours.
- Origin and Destination: Simulated airport codes to represent where the flight begins and ends.
- Price: Current ticket price for the flight.
- Seats Available: The number of seats available on the flight.
- Demand Level: A categorization of expected demand (High, Medium, Low).
- Holiday: Whether the flight is close to a holiday (Yes, No).
- Historical Load Factor: The historical percentage of seats filled.
- Competitor Price: Average price of competitors' flights for similar routes.


In [1]:
import pandas as pd
import numpy as np

def generate_synthetic_dataset(num_records=10000):
    np.random.seed(42)  # For reproducibility

    # Generate dates and times
    start_date = pd.to_datetime('2024-01-01')
    hours_increment = np.random.randint(1, 24, num_records)
    dates = start_date + pd.to_timedelta(hours_increment.cumsum(), unit='h')

    # Generate other features
    day_of_week = dates.dayofweek
    time_of_day = ['Morning' if 5 <= dt.hour < 12 else 'Afternoon' if 12 <= dt.hour < 18 else 'Evening' for dt in dates]
    duration = np.random.uniform(1, 12, num_records)
    origins = ['Origin_' + str(np.random.randint(1, 101)) for _ in range(num_records)]
    destinations = ['Destination_' + str(np.random.randint(1, 101)) for _ in range(num_records)]
    base_price = np.random.uniform(100, 1000, num_records)
    seats_available = np.random.randint(50, 300, num_records)
    demand_factor = np.random.uniform(0.5, 1.5, num_records)
    holiday_effect = [1.5 if dt.month == 12 or dt.month == 7 else 1.0 for dt in dates]
    historical_load_factor = np.random.uniform(0.6, 0.95, num_records)
    competitor_price = base_price * np.random.uniform(0.8, 1.2, num_records)

    # Adjust price based on demand, holiday effect
    prices = base_price * demand_factor * holiday_effect

    # Assemble the dataset
    dataset = pd.DataFrame({
        'Flight ID': range(1, num_records + 1),
        'Date': dates,
        'Day of Week': day_of_week,
        'Time of Day': time_of_day,
        'Duration': duration,
        'Origin': origins,
        'Destination': destinations,
        'Price': prices,
        'Seats Available': seats_available,
        'Historical Load Factor': historical_load_factor,
        'Competitor Price': competitor_price
    })

    return dataset

# Generate the dataset
synthetic_dataset = generate_synthetic_dataset()

# Display the first few rows to verify
print(synthetic_dataset.head())


   Flight ID                Date  Day of Week Time of Day   Duration  \
0          1 2024-01-01 07:00:00            0     Morning   4.279654   
1          2 2024-01-02 03:00:00            1     Evening   2.391107   
2          3 2024-01-02 18:00:00            1     Evening  10.268100   
3          4 2024-01-03 05:00:00            2     Morning   4.983998   
4          5 2024-01-03 13:00:00            2   Afternoon   8.265167   

      Origin     Destination       Price  Seats Available  \
0  Origin_55   Destination_6   95.717913              113   
1  Origin_66  Destination_67  383.451859              186   
2  Origin_60  Destination_10  997.114967              212   
3  Origin_24  Destination_12  408.387064               51   
4  Origin_92  Destination_97  364.175603              292   

   Historical Load Factor  Competitor Price  
0                0.698695        199.030694  
1                0.737009        358.193670  
2                0.699790        855.965979  
3               

# Step 2: Building the Model

In [2]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

In [3]:
# Select features and target
X = synthetic_dataset.drop(['Flight ID', 'Date', 'Price'], axis=1)
y = synthetic_dataset['Price']

In [4]:
# Convert categorical features into one-hot encoded variables
categorical_features = ['Day of Week', 'Time of Day', 'Origin', 'Destination']
one_hot_encoder = OneHotEncoder(handle_unknown='ignore')
preprocessor = ColumnTransformer(transformers=[('cat', one_hot_encoder, categorical_features)], remainder='passthrough')


In [5]:
# Define the model pipeline
model = Pipeline(steps=[('preprocessor', preprocessor),
                        ('regressor', LinearRegression())])


In [6]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [7]:
# Train the model
model.fit(X_train, y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('cat',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  ['Day of Week', 'Time of Day',
                                                   'Origin',
                                                   'Destination'])])),
                ('regressor', LinearRegression())])

In [8]:
# Predict and evaluate
y_pred = model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

In [9]:
print(f"Mean Squared Error: {mse}")
print(f"R-squared: {r2}")

Mean Squared Error: 56768.76543595417
R-squared: 0.5543065023861704


# Saving the Model

In [12]:
import os
from joblib import dump

# Ensure the ./model directory exists
os.makedirs('./model', exist_ok=True)

# Assuming `model` is your trained model pipeline
dump(model, './model/ticket_price_predictor.joblib')


['./model/ticket_price_predictor.joblib']